# TP : Analyse de sentiments dans les critiques de films

## Objectifs

1. Implémenter une manière simple de représenter des données textuelles
2. Implémenter un modèle d'apprentissage statistique basique
3. Utiliser ces représentations et ce modèle pour une tâche d'analyse de sentiments
4. Tenter d'améliorer les résultats avec des outils venus du traitement automatique du langage
5. Comparer les résultats avec une l'implémentation de Scikit-Learn, et avec d'autres méthodes de représentation ou d'apprentissage.

## Dépendances nécessaires

Pour les objectifs 4. et 5., on aura besoin des packages suivants:
- The Machine Learning API Scikit-learn : http://scikit-learn.org/stable/install.html
- The Natural Language Toolkit : http://www.nltk.org/install.html

Les deux sont disponibles avec Anaconda: https://anaconda.org/anaconda/nltk et https://anaconda.org/anaconda/scikit-learn

In [26]:
import os.path as op
import numpy as np

## Charger les données

On récupère les données textuelles dans la variable *texts*

On récupère les labels dans la variable $y$ qui en contient *len(texts)* : $0$ indique que la critique correspondante est négative tandis que $1$ qu'elle est positive.

In [3]:
from glob import glob
filenames_neg = sorted(glob(op.join('.', 'data', 'imdb1', 'neg', '*.txt')))
filenames_pos = sorted(glob(op.join('.', 'data', 'imdb1', 'pos', '*.txt')))

texts_neg = [open(f, encoding="utf8").read() for f in filenames_neg]
texts_pos = [open(f, encoding="utf8").read() for f in filenames_pos]
texts = texts_neg + texts_pos

#Return an array of [1,len(texts)], filled with ones. 
y = np.ones(len(texts), dtype=np.int)
y[:len(texts_neg)] = 0.

print("%d documents" % len(texts))

25000 documents


## Idée principale

On dispose d'une critique étant en fait une liste de mots $s = (w_1, ..., w_N)$, et l'on cherche à trouver la classe associée $c$ - qui dans notre cas, peut-être $c = 0$ ou $c = 1$. L'objectif est donc de trouver pour chaque critique $s$ la classe $\hat{c}$ maximisant la probabilité conditionelle **$P(c|s)$** : 

$$\hat{c} = \underset{c}{\mathrm{argmax}}\, P(c|s) = \underset{c}{\mathrm{argmax}}\,\frac{P(s|c)P(c)}{P(s)}$$

**Hypothèse : P(s) est constante pour chaque classe** :

$$\hat{c} = \underset{c}{\mathrm{argmax}}\,\frac{P(s|c)P(c)}{P(s)} = \underset{c}{\mathrm{argmax}}\,P(s|c)P(c)$$

**Hypothèse naïve : les différentes variables (mots) d'une critique sont indépendantes entre elles** : 

$$P(s|c) = P(w_1, ..., w_N|c)=\Pi_{i=1..N}P(w_i|c)$$

On va donc pouvoir se servir des critiques annotées à notre disposition pour **estimer les probabilités $P(w|c)$ pour chaque mot $w$ étant donné les deux classes $c$**. Ces critiques vont nous permettre d'apprendre à évaluer la "compatibilité" entre les mots et classes.

## Vue générale

### Entraînement: Estimer les probabilités

Pour chaque mot $w$ du vocabulaire $V$, $P(w|c)$ est le nombre d'occurences de $w$ dans une critique ayant pour classe $c$, divisé par le nombre total d'occurences dans $c$. Si on note $T(w,c)$ ce nombre d'occurences, on obtient:

$$P(w|c) = \text{Fréquence de }w\text{ dans }c = \frac{T(w,c)}{\sum_{w' \in V} T(w',c)}$$

### Test: Calcul des scores

Pour faciliter les calculs et éviter les erreurs d'*underflow* et d'approximation, on utilise le "log-sum trick", et on passe l'équation en log-probabilités : 

$$\hat{c} =  \underset{c}{\mathrm{argmax}}\, P(c|s) = \underset{c}{\mathrm{argmax}}\, \left[ \mathrm{log}(P(c)) + \sum_{i=1..N}log(P(w_i|c)) \right]$$

## Représentation adaptée des documents

Notre modèle statistique, comme la plupart des modèles appliqués aux données textuelles, utilise les comptes d'occurences de mots dans un document. Ainsi, une manière très pratique de représenter un document est d'utiliser un vecteur "Bag-of-Words" (BoW), contenant les comptes de chaque mot (indifférement de leur ordre d'apparition) dans le document. 

Si on considère l'ensemble de tous les mots apparaissant dans nos $T$ documents d'apprentissage, que l'on note $V$ (Vocabulaire), on peut créer **un index**, qui est une bijection associant à chaque mot $w$ un entier, qui sera sa position dans $V$. 

Ainsi, pour un document extrait d'un ensemble de documents contenant $|V|$ mots différents, une représentation BoW sera un vecteur de taille $|V|$, dont la valeur à l'indice d'un mot $w$ sera son nombre d'occurences dans le document. 

On peut utiliser la classe **CountVectorizer** de scikit-learn pour mieux comprendre:

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, ClassifierMixin

In [5]:
corpus = ['I walked down down the boulevard', 'I walked down the avenue',
          'I ran down the boulevard', 'I walk down the city','I walk down the the avenue']
vectorizer = CountVectorizer()

Bow = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())
Bow.toarray()

['avenue', 'boulevard', 'city', 'down', 'ran', 'the', 'walk', 'walked']


array([[0, 1, 0, 2, 0, 1, 0, 1],
       [1, 0, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0],
       [1, 0, 0, 1, 0, 2, 1, 0]], dtype=int64)

On affiche d'abord la liste contenant les mots ordonnés selon leur indice (On note que les mots de 2 caractères ou moins ne sont pas pris en compte).

## Détail: entraînement

L'idée est d'extraire le nombre d'occurences $T(w,c)$ de chaque mot $w$ pour chaque classe $c$, ce qui permettra de calculer la matrice de probabilités conditionelles $\pmb{P}$ telle que: $$\pmb{P}_{w,c} = P(w|c)$$

Notons que le nombre d'occurences $T(w,c)$ peut être obtenu facilement à partir des représentations BoW de l'ensemble des documents.

### Procédure:
<img src="algo_train.png" alt="Drawing" style="width: 700px;"/>

## Détail: test

Nous connaissons maintenant les probabilités conditionelles données par la matrice $\pmb{P}$. 
Il faut maintenant obtenir $P(s|c)$ pour le document courant. Cette quantité s'obtient à l'aide d'un calcul simple impliquant la représentation BoW du document et $\pmb{P}$.

### Procédure:
<img src="algo_test.png" alt="Drawing" style="width: 700px;"/>

## Preprocessing du texte: obtenir les représentations BoW

Fonction **à compléter**. Elle renvoie la représentation BoW d'un document.

##### Quelques pointeurs pour les débutants en Python : 

- ```string_1.split(string_2)``` : split the ```string_1``` variable using the ```string_2``` pattern

- ```my_list.append(value)``` : put the variable ```value``` at the end of the list ```my_list```

-  ```words = set()``` : create a set, which is a list of unique values

- ```words.union(my_list)``` : extend the set ```words```

- ```dict(zip(keys, values)))``` : create a dictionnary

- ```for k, text in enumerate(texts)``` : syntax for a loop with the index, ```texts``` begin a list (of texts !)

- ```len(my-list)``` : length of the list ```my_list```


In [6]:
def count_words(texts):
    """Vectorize text : return count of each word in the text snippets

    Parameters
    ----------
    texts : list of str
        The texts

    Returns
    -------
    vocabulary : dict
        A dictionary that points to an index in counts for each word.
    counts : ndarray, shape (n_samples, n_features)
        The counts of each word in each text.
    """
    
    words = set()
    for text in texts:
        words = words.union(set(text.split())) # list of all words
    n_features = len(words) # number of different words
    n_samples = len(texts)
    vocabulary = dict(zip(words, range(n_features))) # vocab[wd] = index ; indexisation
    counts = np.zeros((n_samples, n_features))
    for k, text in enumerate(texts): # enumeration a k for a text[k]
        for w in text.split():
            counts[k][vocabulary[w]] += 1.
    return vocabulary, counts

## Naïve Bayes 

Classe vide : fonctions **à compléter** 
```python
def fit(self, X, y)
``` 
**Entraînement** : va apprendre un modèle statistique basés sur les représentations $X$ correspondant aux labels $y$.

```python
def predict(self, X)
```
**Testing** : va renvoyer les labels prédits par le modèle pour les représentations $X$

##### Quelques pointeurs pour les débutants en Python : 

Utiliser l'API Numpy pour travailler avec des tenseurs


- ```X.shape``` : for a ```numpy.array```, return the dimension of the tensor

- ```np.zeros((dim_1, dim_2,...))``` : create a tensor filled with zeros

- ```np.sum(X, axis = n)``` : sum the tensor over the axis n

- ```np.mean(X, axis = n)```

- ```np.argmax(X, axis = n)```

- ```np.log(X)```

- ```np.dot(X_1, X_1)``` : Matrix multiplication

In [7]:
class NB(BaseEstimator, ClassifierMixin):
    # Les arguments de classe permettent l'héritage de classes de sklearn
    def __init__(self, alpha=1.0):
        # alpha est un paramètre pour le lissage: il correspond à la valeur ligne 10 de l'algorithme d'entraînement
        # Dans l'algorithme d'entraînement, et comme valeur par défaut, on utilise alpha = 1
        self.alpha = alpha

    def fit(self, X, y):
        n_samples, n_features = X.shape

        classes = np.unique(y) # all labels
        n_classes = len(classes) # number of labels

        prior = np.zeros(n_classes)

        tct = np.zeros((n_classes, n_features))
        for k, c in enumerate(classes):
            prior[k] = np.sum(y == c) / float(n_samples)
            tct[k, :] = np.sum(X[y == c, :], axis=0) # number(words|classe)

        alpha = self.alpha  # laplace smoothing / lissage de laplace
        cond_prob = (tct + alpha) / np.sum(tct + alpha, axis=1)[:, None]

        self.prior_ = prior
        self.log_cond_prob_ = np.log(cond_prob)
        return self

    def predict(self, X):
        n_classes = len(self.prior_)
        scores = np.dot(X, self.log_cond_prob_.T)
        scores += np.log(self.prior_)[None, :]
        return np.array([0, 1])[np.argmax(scores, axis=1)]

    def score(self, X, y):
        return np.mean(self.predict(X) == y)

## Expérimentation

On utilise la moitié des données pour l'entraînement, l'autre pour tester le modèle.

In [8]:
# Ici, on part d'un cinquième des données, pour des questions de temps de calcul
texts_red = texts[0::5]
y_red = y [0::5]

print('Nombre de documents:', len(y_red))

Nombre de documents: 5000


In [9]:
voc, X = count_words(texts_red)

In [10]:
nb = NB()
nb.fit(X[::2], y_red[::2])
print(nb.score(X[1::2], y_red[1::2]))

0.8112


## Cross-validation 

Avec la fonction *cross_val_score* de scikit-learn

In [11]:
scores = cross_val_score(nb, X, y_red, cv=5)
print('Score de classification: %s (std %s)' % (np.mean(scores), np.std(scores)))

Score de classification: 0.7872 (std 0.0206242575623948)


## Evaluer les performances: 

**Quelles sont les points forts et les points faibles de ce système ? Comment y remédier ?**

# Pour aller plus loin: 

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

## Scikit-learn

### Améliorer les représentations

On utilise la function 
```python
CountVectorizer
``` 
de scikit-learn pour constituer notre corpus. Elle nous permettra d'améliorer facilement nos représentations BoW.

#### Tf-idf:

Il s'agit du produit de la fréquence du terme (TF) et de sa fréquence inverse dans les documents (IDF).
Cette méthode est habituellement utilisée pour extraire l'importance d'un terme $i$ dans un document $j$ relativement au reste du corpus, à partir d'une matrice d'occurences $mots \times documents$. Ainsi, pour une matrice $\mathbf{T}$ de $|V|$ termes et $D$ documents:
$$\text{TF}(T, w, d) = \frac{T_{w,d}}{\sum_{w'=1}^{|V|} T_{w',d}} $$

$$\text{IDF}(T, w) = \log\left(\frac{D}{|\{d : T_{w,d} > 0\}|}\right)$$

$$\text{TF-IDF}(T, w, d) = \text{TF}(X, w, d) \cdot \text{IDF}(T, w)$$

On peut l'adapter à notre cas en considérant que le contexte du deuxième mot est le document. Cependant, TF-IDF est généralement plus adaptée aux matrices peu denses, puisque cette mesure pénalisera les termes qui apparaissent dans une grande partie des documents. 
    
#### Ne pas prendre en compte les mots trop fréquents:

On peut utiliser l'option 
```python
max_df=1.0
```
pour modifier la quantité de mots pris en compte. 

#### Essayer différentes granularités:

Plutôt que de simplement compter les mots, on peut compter les séquences de mots - de taille limitée, bien sur. 
On appelle une séquence de $n$ mots un $n$-gram: essayons d'utiliser les 2 et 3-grams (bi- et trigrams).
On peut aussi tenter d'utiliser les séquences de caractères à la place de séquences de mots.

On s'intéressera aux options 
```python
analyzer='word'
```
et 
```python
ngram_range=(1, 2)
```
que l'on changera pour modifier la granularité. 

In [13]:
## On peut définir un pipeline que l'on modifiera pour expérimenter.

pipeline_base = Pipeline([
    ('vect', CountVectorizer(analyzer='word', stop_words=None)),
    ('clf', MultinomialNB()),
])
scores = cross_val_score(pipeline_base, texts_red, y_red, cv=5)
print("Classification score: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_tf_idf = Pipeline([
    ('vect', CountVectorizer(analyzer='word', stop_words=None)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
scores = cross_val_score(pipeline_tf_idf, texts_red, y_red, cv=5)
print("Classification score tf-idf: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_maxdf = Pipeline([
    ('vect', CountVectorizer(max_df=0.75, analyzer='word', stop_words=None)),
    ('clf', MultinomialNB()),
])
scores = cross_val_score(pipeline_maxdf, texts_red, y_red, cv=5)
print("Classification score sans mots fréquents: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_bigram = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2), analyzer='word', stop_words=None)),
    ('clf', MultinomialNB()),
])
scores = cross_val_score(pipeline_bigram, texts_red, y_red, cv=5)
print("Classification score bigram: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_trigram = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 3), analyzer='word', stop_words=None)),
    ('clf', MultinomialNB()),
])
scores = cross_val_score(pipeline_trigram, texts_red, y_red, cv=5)
print("Classification score trigram: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_char = Pipeline([
    ('vect', CountVectorizer(ngram_range=(2, 5), analyzer='char', stop_words=None)),
    ('clf', MultinomialNB()),
])
scores = cross_val_score(pipeline_char, texts_red, y_red, cv=5)
print("Classification score char: %s (std %s)",(np.mean(scores), np.std(scores)))

Classification score: %s (std %s) (0.7784000000000001, 0.01426323946374036)
Classification score tf-idf: %s (std %s) (0.7894, 0.024054937123177002)
Classification score sans mots fréquents: %s (std %s) (0.7804, 0.012705904139414883)
Classification score bigram: %s (std %s) (0.8145999999999999, 0.01228983319658974)
Classification score trigram: %s (std %s) (0.8220000000000001, 0.009528903399657253)
Classification score char: %s (std %s) (0.7836000000000001, 0.01611955334368792)


### Natural Language Toolkit (NLTK)

### Stemming 

Permet de revenir à la racine d'un mot: on peut ainsi grouper différents mots autour de la même racine, ce qui facilite la généralisation. Utiliser:
```python
from nltk import SnowballStemmer
```

In [14]:
from nltk import SnowballStemmer
stemmer = SnowballStemmer("english")

#### Exemple d'utilisation:

In [15]:
words = ['singers', 'cat', 'generalization', 'philosophy', 'psychology', 'philosopher']
for word in words:
    print('word : %s ; stemmed : %s' %(word, stemmer.stem(word)))#.decode('utf-8'))))

word : singers ; stemmed : singer
word : cat ; stemmed : cat
word : generalization ; stemmed : general
word : philosophy ; stemmed : philosophi
word : psychology ; stemmed : psycholog
word : philosopher ; stemmed : philosoph


#### Transformation des données:

Classe vide : function **à compléter** 
```python
def stem(X)
``` 

In [16]:
def stem(X): 
    X_stem = []
    for text in X:
        wordlist = []
        for word in text.split(" "):
            stemmword = stemmer.stem(word)#.decode('utf-8'))
            wordlist.append(stemmword)
        X_stem.append(' '.join(wordlist))
    return X_stem

In [17]:
texts_stemmed = stem(texts_red)
voc, X = count_words(texts_stemmed)
nb = NB()

scores = cross_val_score(nb, X, y_red, cv=5)
print('Score de classification: %s (std %s)' % (np.mean(scores), np.std(scores)))

Score de classification: 0.7882 (std 0.015011995203836177)


### Partie du discours

Pour généraliser, on peut aussi utiliser les parties du discours (Part of Speech, POS) des mots, ce qui nous permettra  de filtrer l'information qui n'est potentiellement pas utile au modèle. On va récupérer les POS des mots à l'aide des fonctions:
```python
from nltk import pos_tag, word_tokenize
```

In [18]:
import nltk
from nltk import pos_tag, word_tokenize

#### Exemple d'utilisation:

In [21]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

pos_tag(word_tokenize(('I am Sam')))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Matthieu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


[('I', 'PRP'), ('am', 'VBP'), ('Sam', 'NNP')]

Détails des significations des tags POS: https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk

####  Transformation des données:

Classe vide : fonction **à compléter** 
```python
def pos_tag_filter(X, good_tags=['NN', 'VB', 'ADJ', 'RB'])
``` 

Ne garder que les noms, adverbes, verbes et adjectifs pour notre modèle. 

In [22]:
def pos_tag_filter(X, good_tags=['NN', 'VB', 'ADJ', 'RB']):
    X_pos = []
    for text in X:
        tags = nltk.pos_tag(nltk.word_tokenize(text))
        X_pos.append(' '.join([w for w, t in tags if t in good_tags]))
    return X_pos

In [23]:
texts_POS = pos_tag_filter(texts_red)
voc, X = count_words(texts_POS)
nb = NB()

scores = cross_val_score(nb, X, y_red, cv=5)
print('Score de classification: %s (std %s)' % (np.mean(scores), np.std(scores)))

Score de classification: 0.7556 (std 0.012419339757008029)


### Stop-words 

Les "stop-words" sont les mots apparaissant fréquemment dans les données et que l'on juge non représentatifs. On les considère comme du bruit. Une liste de stop-words est disponible dans le fichier *english.stop*

In [24]:
def readFile(fileName):
    """
     * Code for reading a file.  you probably don't want to modify anything here, 
     * unless you don't like the way we segment files.
    """
    contents = []
    f = open(fileName)
    for line in f:
        contents.append(line)
    f.close()
    result = ('\n'.join(contents)).split() 
    return result

sw = readFile('english.stop')
sw[0:50]

['a',
 "a's",
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 "aren't",
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'associated']

####  Transformation des données:

Classe vide : fonction **à compléter** 
```python
def filterStopWords(X)
``` 

In [29]:
def filterStopWords(X):
    """Filters stop words."""
    X_filtered = []
    for text in X:
        text_filtered = [w for w in text.split() if w not in sw]
        X_filtered.append(' '.join(text_filtered))
    return X_filtered

In [30]:
texts_stop = filterStopWords(texts_red)
voc, X = count_words(texts_stop)
nb = NB()

scores = cross_val_score(nb, X, y_red, cv=5)
print('Score de classification: %s (std %s)' % (np.mean(scores), np.std(scores)))

Score de classification: 0.7804 (std 0.023052982453470074)


### Bonus: Utilisation d'un classifieur plus complexe ?

On peut utiliser les implémentations scikit-learn de classifieurs moins naïfs, comme la régression logistique ou les SVM. 

In [56]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [58]:
pipeline_logistic = Pipeline([
    ('vect', CountVectorizer(max_df=0.75, ngram_range=(1, 3),
                             analyzer='word', stop_words=None)),
    ('clf', LogisticRegression(C=1.)),
])
scores = cross_val_score(pipeline_logistic, texts_red, y_red, cv=5)
print("Classification score: %s (std %s)" % (np.mean(scores), np.std(scores)))

pipeline_svm = Pipeline([
    ('vect', CountVectorizer(max_df=0.75, ngram_range=(1, 3),
                             analyzer='word', stop_words=None)),
    ('clf', LinearSVC(C=1.)),
])
scores = cross_val_score(pipeline_svm, texts_red, y_red, cv=5)
print("Classification score: %s (std %s)" % (np.mean(scores), np.std(scores)))

/disk/ocean/mlabeau/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Classification score: 0.8459999999999999 (std 0.006928203230275515)


/disk/ocean/mlabeau/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Classification score: 0.8427999999999999 (std 0.006615134163416496)
